## reading the dataset we will work with

In [1]:
import pandas as pd
import random as rand

In [2]:
import numpy as np
import torch.nn as nn

In [3]:
data=pd.read_csv("/kaggle/input/ctu-13/CTU_13.csv")

/tmp/ipykernel_36/3032590272.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("/kaggle/input/ctu-13/CTU_13.csv")


In [4]:
lst = []
for i in data['Label']:
    if 'Botnet' in i:
        lst.append(1)
    elif 'Normal' in i:
        lst.append(2)
    else:
        lst.append(0)

data['Label']=lst

we remove the background data points

In [5]:
data=data[data['Label']!=0]

In [6]:
direction_number = []
for i in data['Dir']:
    if i == '  <->':
        direction_number.append(1)
    elif i == '   ->':
        direction_number.append(2)
    else:
        direction_number.append(0)

data['Dir'] = direction_number

In [7]:
len(data)

801132

In [8]:
#the unlabeled version we will use for training
oldd=data.drop(columns=['Unnamed: 0'],inplace=False)
#dataset=data.drop(columns=['Label','Unnamed: 0'],inplace=False)
dataset=data.drop(columns=['Unnamed: 0'],inplace=False)

In [9]:
dataset

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label
150,2011/08/10 09:46:53.160043,0.000000,udp,147.32.80.9,53,2,147.32.86.111,54230,INT,0.0,NaN,1,141,141,2
408,2011/08/10 09:46:54.045343,0.000000,udp,147.32.80.9,53,2,147.32.86.111,58314,INT,0.0,NaN,1,247,247,2
495,2011/08/10 09:46:54.460552,0.000000,udp,147.32.80.9,53,2,147.32.86.111,54823,INT,0.0,NaN,1,173,173,2
596,2011/08/10 09:46:54.779125,0.000291,udp,147.32.84.134,40693,1,147.32.80.9,53,CON,0.0,0.0,2,207,72,2
597,2011/08/10 09:46:54.779641,0.000321,udp,147.32.84.134,47415,1,147.32.80.9,53,CON,0.0,0.0,2,247,72,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19975942,2011/08/16 09:35:46.594868,0.000429,udp,147.32.84.170,55989,1,147.32.80.9,53,CON,0.0,0.0,2,400,74,2
19975943,2011/08/16 09:35:46.596051,0.018334,tcp,147.32.84.170,43453,2,209.85.148.99,80,FSA_FSA,0.0,0.0,6,412,272,2
19976516,2011/08/16 09:35:56.593152,0.000521,udp,147.32.84.170,38927,1,147.32.80.9,53,CON,0.0,0.0,2,400,74,2
19976517,2011/08/16 09:35:56.594273,0.000242,udp,147.32.84.170,37057,1,147.32.80.9,53,CON,0.0,0.0,2,400,74,2


In [10]:
dataset.columns

Index(['StartTime', 'Dur', 'Proto', 'SrcAddr', 'Sport', 'Dir', 'DstAddr',
       'Dport', 'State', 'sTos', 'dTos', 'TotPkts', 'TotBytes', 'SrcBytes',
       'Label'],
      dtype='object')

In [11]:
dataset.dropna(inplace=True)

In [12]:
dt = pd.to_datetime(dataset['StartTime'], format='%Y/%m/%d %H:%M:%S.%f')
dt

596        2011-08-10 09:46:54.779125
597        2011-08-10 09:46:54.779641
598        2011-08-10 09:46:54.780268
1080       2011-08-10 09:46:56.955681
1269       2011-08-10 09:46:57.796719
                      ...            
19975942   2011-08-16 09:35:46.594868
19975943   2011-08-16 09:35:46.596051
19976516   2011-08-16 09:35:56.593152
19976517   2011-08-16 09:35:56.594273
19976518   2011-08-16 09:35:56.595551
Name: StartTime, Length: 563790, dtype: datetime64[ns]

In [13]:
df = pd.DataFrame({'datetime': dt})


In [14]:
dataset['year'] = df['datetime'].dt.year
dataset['month'] = df['datetime'].dt.month
dataset['day'] = df['datetime'].dt.day
dataset['hour'] = df['datetime'].dt.hour
dataset['minute'] = df['datetime'].dt.minute
dataset['second'] = df['datetime'].dt.second
dataset['dayofweek'] = df['datetime'].dt.dayofweek  # Monday=0, Sunday=6
dataset['is_weekend'] = dataset['dayofweek'].isin([5,6]).astype(int)

In [15]:
dataset.drop(columns=['StartTime'],inplace=True)

In [16]:
dataset

,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,...,SrcBytes,Label,year,month,day,hour,minute,second,dayofweek,is_weekend
596,0.000291,udp,147.32.84.134,40693,1,147.32.80.9,53,CON,0.0,0.0,...,72,2,2011,8,10,9,46,54,2,0
597,0.000321,udp,147.32.84.134,47415,1,147.32.80.9,53,CON,0.0,0.0,...,72,2,2011,8,10,9,46,54,2,0
598,0.007678,tcp,147.32.84.134,45718,2,88.86.100.176,80,FSPA_FSPA,0.0,0.0,...,457,2,2011,8,10,9,46,54,2,0
1080,3585.430176,tcp,147.32.84.164,39177,0,74.125.232.216,443,PA_PA,0.0,0.0,...,219986,2,2011,8,10,9,46,56,2,0
1269,0.000318,udp,147.32.84.134,36480,1,147.32.80.9,53,CON,0.0,0.0,...,72,2,2011,8,10,9,46,57,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19975942,0.000429,udp,147.32.84.170,55989,1,147.32.80.9,53,CON,0.0,0.0,...,74,2,2011,8,16,9,35,46,1,0
19975943,0.018334,tcp,147.32.84.170,43453,2,209.85.148.99,80,FSA_FSA,0.0,0.0,...,272,2,2011,8,16,9,35,46,1,0
19976516,0.000521,udp,147.32.84.170,38927,1,147.32.80.9,53,CON,0.0,0.0,...,74,2,2011,8,16,9,35,56,1,0
19976517,0.000242,udp,147.32.84.170,37057,1,147.32.80.9,53,CON,0.0,0.0,...,74,2,2011,8,16,9,35,56,1,0


In [17]:
dataset.dtypes

Dur           float64
Proto          object
SrcAddr        object
Sport          object
Dir             int64
DstAddr        object
Dport          object
State          object
sTos          float64
dTos          float64
TotPkts         int64
TotBytes        int64
SrcBytes        int64
Label           int64
year            int32
month           int32
day             int32
hour            int32
minute          int32
second          int32
dayofweek       int32
is_weekend      int64
dtype: object

In [18]:
cat_cols=['Proto','SrcAddr','DstAddr','Dport','State']

In [19]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


In [20]:
for col in cat_cols:
    print(col)
    numeric_labels = le.fit_transform(dataset[col])
    dataset[col]=numeric_labels

    
    
    

Proto
SrcAddr
DstAddr
Dport
State


In [21]:
numeric_labels = le.fit_transform(dataset['Sport'].astype('str'))


In [22]:
dataset['Sport']=numeric_labels


In [23]:
print(int(len(dataset)*0.8)) #for training encider for embedings
print(int(len(dataset)*0.1)) #for trainnig linear model later
print(int(len(dataset)*0.8)) #for evaluation 

451032
56379
451032


In [24]:
datasetlabeled=dataset.copy()


In [25]:
datasetlabeled.columns

Index(['Dur', 'Proto', 'SrcAddr', 'Sport', 'Dir', 'DstAddr', 'Dport', 'State',
       'sTos', 'dTos', 'TotPkts', 'TotBytes', 'SrcBytes', 'Label', 'year',
       'month', 'day', 'hour', 'minute', 'second', 'dayofweek', 'is_weekend'],
      dtype='object')

In [26]:
dataset.drop(columns=['Label'],inplace=True)

### we preserve some of the data from trainin the linear

In [27]:
total_len = len(dataset)

# 80% for encoder (unlabeled)
encoder_data = dataset[:int(total_len * 0.8)]

# Next 10% for linear classifier training (labeled)
linear_model = datasetlabeled[int(total_len * 0.8):int(total_len * 0.9)]

# Final 10% for evaluation (labeled)
datasetevaluation = datasetlabeled[int(total_len * 0.9):]
dataset=encoder_data

In [28]:
linear_model.columns

Index(['Dur', 'Proto', 'SrcAddr', 'Sport', 'Dir', 'DstAddr', 'Dport', 'State',
       'sTos', 'dTos', 'TotPkts', 'TotBytes', 'SrcBytes', 'Label', 'year',
       'month', 'day', 'hour', 'minute', 'second', 'dayofweek', 'is_weekend'],
      dtype='object')

In [29]:
dataset.columns

Index(['Dur', 'Proto', 'SrcAddr', 'Sport', 'Dir', 'DstAddr', 'Dport', 'State',
       'sTos', 'dTos', 'TotPkts', 'TotBytes', 'SrcBytes', 'year', 'month',
       'day', 'hour', 'minute', 'second', 'dayofweek', 'is_weekend'],
      dtype='object')

In [30]:
dataset

,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,...,TotBytes,SrcBytes,year,month,day,hour,minute,second,dayofweek,is_weekend
596,0.000291,3,2,23372,1,1366,752,4,0.0,0.0,...,207,72,2011,8,10,9,46,54,2,0
597,0.000321,3,2,30772,1,1366,752,4,0.0,0.0,...,247,72,2011,8,10,9,46,54,2,0
598,0.007678,2,2,28903,2,9130,902,37,0.0,0.0,...,1129,457,2011,8,10,9,46,54,2,0
1080,3585.430176,2,6,21702,0,7510,665,62,0.0,0.0,...,509187,219986,2011,8,10,9,46,56,2,0
1269,0.000318,3,2,18733,1,1366,752,4,0.0,0.0,...,207,72,2011,8,10,9,46,57,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16123493,0.453756,3,15,11330,1,1366,752,4,0.0,0.0,...,274,73,2011,8,17,16,48,47,2,0
16123494,0.000465,3,15,11320,1,1366,752,4,0.0,0.0,...,394,73,2011,8,17,16,48,47,2,0
16123496,0.140354,3,10,5039,1,1366,752,4,0.0,0.0,...,246,77,2011,8,17,16,48,47,2,0
16123499,124.327148,2,7,33538,2,2503,902,37,0.0,0.0,...,7519,733,2011,8,17,16,48,47,2,0


## Data augmentation module

since here we are working with rows in a tabular data , in this case from the ctu13 dataset

The task is to create two different but semantically simliar verisons of the same og flow

Two altered versions from the og flow

In [ ]:
augmented1 = dataset.copy()
nbrows=len(augmented1)

In [ ]:
# set random features to mean val ==feature masking
maskrate=0.1 # for each eature we want to mask 10% of the rows
for col in augmented1.columns:
    meanval=augmented1[col].mean()
    maskindexes=rand.sample(range(nbrows),int(maskrate*nbrows))
    augmented1.loc[maskindexes,col]=meanval

In [ ]:
#add gaussian noise
for i in range(0,nbrows):
    noise=np.random.normal(0, 0.1, augmented1[i].shape)
    augmented1[i]+=noise

must be applied to all rows , mean0 and std 0.1

In [ ]:
#Feature dropping
for col in augmented2.columns:
    dropindexes=rand.sample(range(nbrows),int(maskrate*nbrows))
    augmented2.loc[dropindexes,col]=None

### For SimCLR-style training, both views (versions) should be augmented versions of the same original sample , the concept is the model sees two different views of the same data so we will create augmented2 that do other alterations 

In [ ]:
augmented2=dataset.copy()

In [ ]:
import numpy as np

In [31]:
def augment_dataset(dataset,maskrate,noisestd,droprate):
     augmented = dataset.copy()
     nbrows=len(augmented)
    #feature masking
     for col in augmented.columns:
         meanval=augmented[col].mean()
         maskindexes=rand.sample(list(augmented.index), int(maskrate * nbrows))
         augmented.loc[maskindexes,col]=meanval
    #noise
     noise=np.random.normal(0,noisestd,augmented.shape)
     augmented+=noise
    #feature dropping
     """for col in augmented.columns:
         dropindex=rand.sample(list(augmented.index),int(droprate*nbrows))
         augmented.loc[dropindex,col]=np.nan"""
     return augmented

    

In [32]:
augmented1 = augment_dataset(dataset, maskrate=0.1, noisestd=0.1, droprate=0)


/tmp/ipykernel_36/1180384435.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.687099806665602' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  augmented.loc[maskindexes,col]=meanval
/tmp/ipykernel_36/1180384435.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '8.84473607194168' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  augmented.loc[maskindexes,col]=meanval
/tmp/ipykernel_36/1180384435.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '22898.525774224447' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  augmented.loc[maskindexes,col]=meanval
/tmp/ipykernel_36/1180384435.py:8: FutureWarning: Setting an item o

In [33]:
augmented1

,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,...,TotBytes,SrcBytes,year,month,day,hour,minute,second,dayofweek,is_weekend
596,-0.056043,3.055470,2.206867,23372.036321,0.730208,2652.319078,752.029289,3.937152,0.071304,-0.022586,...,206.958620,72.036475,2010.970417,7.943518,10.030266,9.027731,45.910215,53.894109,2.085421,0.122742
597,0.246827,2.810577,8.808057,30771.732071,1.189592,1366.106159,752.045051,4.068776,-0.126208,0.025849,...,247.085734,71.990532,2011.011093,7.860659,10.079607,8.925192,45.913404,54.163410,1.981382,-0.089223
598,0.006127,2.099980,2.074366,28902.964108,1.336987,9130.053857,901.919363,37.013094,0.069937,-0.122148,...,1129.110299,456.931496,2010.801718,8.052058,10.037998,13.419095,45.911184,54.126565,1.971621,0.061325
1080,3585.495918,2.214296,6.035004,21702.080500,-0.138863,7509.896480,665.156491,62.074909,0.050984,-0.153234,...,509186.869611,219986.060031,2010.958553,8.095231,9.896014,9.069727,45.889059,55.916231,2.005165,-0.071901
1269,-0.029892,2.706648,8.858157,18733.080155,0.853658,1365.947978,751.956388,3.991726,-0.072392,-0.066707,...,206.950474,72.196326,2010.969474,8.035576,9.892865,9.049450,45.985238,57.131805,1.944175,0.105504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16123493,0.493058,3.090331,8.976516,11329.901761,1.292325,1365.850975,772.154995,4.006896,0.049794,-0.076875,...,13542.406410,72.860647,2011.002735,8.051521,16.885862,15.890759,47.989948,46.867800,1.947492,0.364655
16123494,-0.025360,3.033929,15.076222,11319.952682,0.859379,1366.186071,751.892379,4.067149,-0.097371,-0.077113,...,394.256220,1783.441329,2011.086883,7.960079,16.980798,16.028659,28.726523,47.133039,2.404429,0.008372
16123496,-0.044345,3.014969,8.955661,5038.786797,1.045429,1366.050931,751.941707,3.878404,0.188913,0.010263,...,246.017383,77.030546,2011.028623,8.007716,16.963190,13.131696,47.982752,47.112391,1.940738,0.081698
16123499,124.107073,1.966222,6.960711,22898.413329,2.019077,2502.947568,901.801248,37.128673,0.069636,-0.017601,...,7519.048854,1783.481461,2010.952506,7.994147,17.193424,15.942844,48.037815,46.842989,1.942501,0.174298


In [34]:
augmented2 = augment_dataset(dataset, maskrate=0.11, noisestd=0.12, droprate=0.1)


/tmp/ipykernel_36/1180384435.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.687099806665602' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  augmented.loc[maskindexes,col]=meanval
/tmp/ipykernel_36/1180384435.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '8.84473607194168' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  augmented.loc[maskindexes,col]=meanval
/tmp/ipykernel_36/1180384435.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '22898.525774224447' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  augmented.loc[maskindexes,col]=meanval
/tmp/ipykernel_36/1180384435.py:8: FutureWarning: Setting an item o

In [35]:
print(augmented2.isnull().sum())
print(augmented1.isnull().sum())

Dur           0
Proto         0
SrcAddr       0
Sport         0
Dir           0
DstAddr       0
Dport         0
State         0
sTos          0
dTos          0
TotPkts       0
TotBytes      0
SrcBytes      0
year          0
month         0
day           0
hour          0
minute        0
second        0
dayofweek     0
is_weekend    0
dtype: int64
Dur           0
Proto         0
SrcAddr       0
Sport         0
Dir           0
DstAddr       0
Dport         0
State         0
sTos          0
dTos          0
TotPkts       0
TotBytes      0
SrcBytes      0
year          0
month         0
day           0
hour          0
minute        0
second        0
dayofweek     0
is_weekend    0
dtype: int64


In [36]:
augmented2.dropna(inplace=True)

## step2 encoding with an MLP

(batch_size, nb_features) the size of the input , we go with batch size 32 and nbfeatures here is  21

In [37]:
len(augmented1.columns) #21
augmented1.columns

Index(['Dur', 'Proto', 'SrcAddr', 'Sport', 'Dir', 'DstAddr', 'Dport', 'State',
       'sTos', 'dTos', 'TotPkts', 'TotBytes', 'SrcBytes', 'year', 'month',
       'day', 'hour', 'minute', 'second', 'dayofweek', 'is_weekend'],
      dtype='object')

In [38]:
import torch.nn as nn

In [39]:
class SimCLRMLP(nn.Module):
    def __init__(self,hidden1=256,hidden2=128,inputsize=21,outputsize=64):
        super(SimCLRMLP, self).__init__()
        self.fc1=nn.Linear(inputsize,hidden1)
        self.fc2=nn.Linear(hidden1,hidden2)
        self.fc3=nn.Linear(hidden2,outputsize)
        self.activation=nn.ReLU()
    def forward(self,x):
        x=self.activation(self.fc1(x))
        x=self.activation(self.fc2(x))
        x=self.fc3(x)
        return x

In [40]:
mlp=SimCLRMLP()

the mlp is just to go from tabular data to a vecotr embedding , then we use a projection hear so we are in the space we will do our constructive loss in 

## step3 projection layer

In [41]:
class ProjectionMLP(nn.Module):
    def __init__(self,inputsize=64,hiddensize=128,outputsize=128):
        super(ProjectionMLP,self).__init__()
        self.fc1=nn.Linear(inputsize,hiddensize)
        self.fc2=nn.Linear(hiddensize,outputsize)
        self.relu=nn.ReLU()

    def forward(self,x):
        x=self.relu(self.fc1(x))
        x=self.fc2(x)
        return x
        
        


### Data loader

In [42]:
import torch
from torch.utils.data import DataLoader,TensorDataset
print(type(dataset),dataset.shape)

<class 'pandas.core.frame.DataFrame'> (451032, 21)


In [43]:
X_tensor = torch.tensor(dataset.values, dtype=torch.float32) 

In [44]:
flow_dataset = TensorDataset(X_tensor) 

In [45]:
dataloader = DataLoader(flow_dataset, batch_size=256, shuffle=True)


In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = SimCLRMLP().to(device)
projection_head = ProjectionMLP(inputsize=64).to(device)


In [47]:
optimizer = torch.optim.Adam(list(encoder.parameters()) + list(projection_head.parameters()), lr=1e-3)


In [48]:
print(len(dataset))

451032


In [ ]:
def nt_xent_loss(z1, z2, temperature=0.5):
    """
    Normalized Temperature-scaled Cross Entropy Loss (NT-Xent) for SimCLR.
    """
    batch_size = z1.size(0)

    z1 = F.normalize(z1, dim=1)
    z2 = F.normalize(z2, dim=1)

    z = torch.cat([z1, z2], dim=0)  # (2N, dim)

    sim_matrix = F.cosine_similarity(z.unsqueeze(1), z.unsqueeze(0), dim=2)  # (2N, 2N)

    mask = torch.eye(2 * batch_size, dtype=torch.bool).to(z.device)
    sim_matrix = sim_matrix[~mask].view(2 * batch_size, -1)

    positives = torch.cat([torch.arange(batch_size, 2*batch_size),
                           torch.arange(0, batch_size)]).to(z.device)

    pos_sim = torch.cat([F.cosine_similarity(z1, z2),
                         F.cosine_similarity(z2, z1)])

    numerator = torch.exp(pos_sim / temperature)
    denominator = torch.sum(torch.exp(sim_matrix / temperature), dim=1)

    loss = -torch.log(numerator / denominator)
    return loss.mean()


nt_xent loss : type of constractive loss == we bring together simliar things and push away non sim ?!
NT-Xent loss (Normalized Temperature-scaled Cross Entropy Loss)
used in self supervised learning

how it works
the pos pais are the two views we make
everything else is neg pairs
the model that way learn the embedding of view1 and view 2 are close

In [50]:
import torch.nn.functional as F


## training loop and constractive loss

In [ ]:
epochs = 20
for epoch in range(epochs):
    total_loss = 0
    for batch in dataloader:
        batch_data = batch[0].to(device)
        
        batch_np = batch_data.cpu().numpy()
        view1_np = augment_dataset(pd.DataFrame(batch_np), maskrate=0.1, noisestd=0.01, droprate=0.05).values
        view2_np = augment_dataset(pd.DataFrame(batch_np), maskrate=0.1, noisestd=0.01, droprate=0.05).values
        #idk whre the nan came from 
        view1_np = np.nan_to_num(view1_np, nan=0.0, posinf=1e6, neginf=-1e6)

       #   print(np.isnan(view1_np).any(), np.isinf(view1_np).any())

        view1 = torch.tensor(view1_np, dtype=torch.float32).to(device)
        view2 = torch.tensor(view2_np, dtype=torch.float32).to(device)
        
        h1 = encoder(view1)
        z1 = projection_head(h1)
        z1 = F.normalize(z1, dim=1)  
        
        h2 = encoder(view2)
        z2 = projection_head(h2)
        z2 = F.normalize(z2, dim=1)
        
        #  NT-Xent loss
        loss = nt_xent_loss(z1, z2)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

Epoch [1/20], Loss: 4.8526
Epoch [2/20], Loss: 4.7096
Epoch [3/20], Loss: 4.6685
Epoch [4/20], Loss: 4.6409
Epoch [5/20], Loss: 4.6233
Epoch [6/20], Loss: 4.6087
Epoch [7/20], Loss: 4.5966
Epoch [8/20], Loss: 4.5889
Epoch [9/20], Loss: 4.5799
Epoch [10/20], Loss: 4.5751
Epoch [11/20], Loss: 4.5702
Epoch [12/20], Loss: 4.5659
Epoch [13/20], Loss: 4.5636
Epoch [14/20], Loss: 4.5606
Epoch [15/20], Loss: 4.5593
Epoch [16/20], Loss: 4.5569
Epoch [17/20], Loss: 4.5554
Epoch [18/20], Loss: 4.5538
Epoch [19/20], Loss: 4.5534
Epoch [20/20], Loss: 4.5499


In [52]:
print("uh?")

uh?


what we have to understand here is the position of slimclr in this proj or any other use , it isnt the main model that will be doing the classification job  , what it serves as is a way to train the model on unlabeled data and makes it understand the embedding and able to generate the accurate embedding , the linear classfier we bring in the game later , will not use the data direclty but the embedding that the trained encoder will generate holly molly that is cool

we discard the mlp cause we only needed it to get the constractive loss , we when usage now use only the encoder that will get us the right embeddings

### freese the encoder

In [55]:
encoder = encoder.to(device)  # make sure encoder is on GPU if available


In [56]:
encoder.eval()
projection_head.eval()  #wont be used

for param in encoder.parameters():
    param.requires_grad = False #freeze it upp

# we define the lieanr classifier

In [57]:
import torch.nn as nn

embedding_dim = 64  
num_classes = 2  
linear_classifier = nn.Linear(embedding_dim, num_classes)

### we train the linear classifire

In [58]:
X_labeled=linear_model.drop(columns=['Label'],inplace=False)
y_labels=linear_model['Label']

In [59]:
y_labels.unique() #the 1 2 wont work with crossentrloss

array([1, 2])

In [70]:
y_labels = torch.tensor(y_labels.values, dtype=torch.long)


In [72]:
unique_labels = torch.unique(y_labels)
label_map = {old.item(): new for new, old in enumerate(unique_labels)}
y_labels = torch.tensor([label_map[int(l)] for l in y_labels], dtype=torch.long)

In [73]:
y_labels

tensor([0, 0, 0,  ..., 1, 0, 0])

In [74]:
X_tensor = torch.tensor(X_labeled.values, dtype=torch.float32).to(device)
#y_tensor = torch.tensor(y_labels.values, dtype=torch.long).to(device)

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [65]:
with torch.no_grad():
    embeddings = encoder(X_tensor)

In [80]:
def makedataloadder(dataset):#for the dataset with label
        X_labeled=dataset.drop(columns=['Label'],inplace=False)
        y_labels=dataset['Label']
        y_labels = torch.tensor(y_labels.values, dtype=torch.long)
        unique_labels = torch.unique(y_labels)
        label_map = {old.item(): new for new, old in enumerate(unique_labels)}
        y_labels = torch.tensor([label_map[int(l)] for l in y_labels], dtype=torch.long)
        X_tensor = torch.tensor(X_labeled.values, dtype=torch.float32).to(device)
        with torch.no_grad():
            embeddings = encoder(X_tensor)
        clf_dataset = TensorDataset(embeddings, y_labels)
        clf_loader = DataLoader(clf_dataset, batch_size=256, shuffle=True)
        return clf_loader



In [ ]:
clf_dataset = TensorDataset(embeddings, y_labels)
clf_loader = DataLoader(clf_dataset, batch_size=256, shuffle=True)

# da classifier
classifier = nn.Linear(embeddings.shape[1], len(unique_labels))
classifier = classifier.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)

#  classifier training loop
for epoch in range(20):
    total_loss = 0
    for xb, yb in clf_loader:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        preds = classifier(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(clf_loader):.4f}")

Epoch 1, Loss: 148.6328
Epoch 2, Loss: 46.1276
Epoch 3, Loss: 24.6642
Epoch 4, Loss: 22.1683
Epoch 5, Loss: 23.8489
Epoch 6, Loss: 28.9456
Epoch 7, Loss: 21.2504
Epoch 8, Loss: 16.9716
Epoch 9, Loss: 17.6823
Epoch 10, Loss: 17.4710
Epoch 11, Loss: 26.7376
Epoch 12, Loss: 16.4018
Epoch 13, Loss: 16.1348
Epoch 14, Loss: 36.5545
Epoch 15, Loss: 36.2790
Epoch 16, Loss: 16.5747
Epoch 17, Loss: 13.4930
Epoch 18, Loss: 22.3170
Epoch 19, Loss: 12.3014
Epoch 20, Loss: 36.5032


### Evaluate this bad boy

In [78]:
datasetevaluation

,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,...,SrcBytes,Label,year,month,day,hour,minute,second,dayofweek,is_weekend
18322351,0.189872,3,7,2834,1,1366,752,4,0.0,0.0,...,74,1,2011,8,15,18,47,41,0,0
18322356,0.189283,3,7,7717,1,1366,752,4,0.0,0.0,...,69,1,2011,8,15,18,47,41,0,0
18322408,0.030104,3,8,34660,1,1366,752,4,0.0,0.0,...,82,2,2011,8,15,18,47,41,0,0
18322409,0.000234,3,8,19210,1,1366,752,4,0.0,0.0,...,82,2,2011,8,15,18,47,41,0,0
18322410,0.000239,3,8,26579,1,1366,752,4,0.0,0.0,...,82,2,2011,8,15,18,47,41,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19975942,0.000429,3,8,40166,1,1366,752,4,0.0,0.0,...,74,2,2011,8,16,9,35,46,1,0
19975943,0.018334,2,8,26411,2,4234,902,30,0.0,0.0,...,272,2,2011,8,16,9,35,46,1,0
19976516,0.000521,3,8,21426,1,1366,752,4,0.0,0.0,...,74,2,2011,8,16,9,35,56,1,0
19976517,0.000242,3,8,19367,1,1366,752,4,0.0,0.0,...,74,2,2011,8,16,9,35,56,1,0


In [83]:
evaldfloader=makedataloadder(datasetevaluation)
evaldfloader

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

def evaluate(encoder, classifier, dataloader, device):
    encoder.eval()
    classifier.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)


            #  classifier outputs
            logits = classifier(X_batch)
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds)

    print(f"Accuracy: {acc:.4f}")
    print("Classification Report:\n", report)

    return acc, report


In [85]:
acc, report = evaluate(encoder, classifier, evaldfloader, device)


Accuracy: 0.9204
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.97      0.92     27352
           1       0.96      0.88      0.92     29027

    accuracy                           0.92     56379
   macro avg       0.92      0.92      0.92     56379
weighted avg       0.92      0.92      0.92     56379



In [ ]:
#make dataset with labeled data
#get loss function
#finsish triannig loop
#evaluation loop